# Lab5 - Part 2: Using a deployed Watson Machine Learning model from a python notebook

This is a 'stretch' lab to complement the Watson ML model training and deployment, we show how to invoke the deployed Watson ML scoring from a Jupyter Notebook

### Gathering information from the WML environment and deployed model
You will have to get out of this notebook, possibly in a separate tab, to gather the following information

#### Watson Machine Learning Credentials:
* You will need to retrieve your ML service credentials:
  * These are available form the Watson Data Platform `Services/Compute Services` menu
  * Selecting `Manage in IBM  Cloud` will take you to the service page where you can copy the credentials from
* Paste the credentials in the code cell below and execute it

In [1]:
### REPLACE the code below with your own credentials placed in the wml_credentials variable

wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "xwcd7ZElMJKAlYE+6a+2mPomD93xr18uElxnbQIJrz9sO0XEfzD83V0rZ42T8V7aHxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "b345d2c1-577d-4bad-bae4-627c50cc8a87",
  "password": "b8fbcd21-b77f-4840-8109-1c3b660ec2af",
  "instance_id": "f8b2d7ff-25d2-4ae6-bd84-ffafcda86acb"
}

#### Watson Machine Learning deployed model boilerplate code:
* Navigate back to you Project's Assets, down to `Models` section
* Open the deployed model, navigate to the `Implementation` tab
* Copy the Python sample code from your Deployed ML instance, and paste to _partially_ **replace** the code cell below:
  * Comment-out or remove the wml_credentials={....} lines, as they are already set in the cell above
  * Set the `payload_scoring` back to: 
``` python
payload_scoring = {"fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION"], "values": [["M",23,"Married","Professional"],["F",27,"Single","Professional"]]}
```
* Execute the code cell, this will run the scoring against the two candidates specified in `payload_scoring`

In [2]:
import urllib3, requests, json

# retrieve your wml_service_credentials_username, wml_service_credentials_password, and wml_service_credentials_url from the
# Service credentials associated with your IBM Cloud Watson Machine Learning Service instance

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION"], "values": [["M",23,"Married","Professional"],["F",27,"Single","Professional"]]}

response_scoring = requests.post('https://ibm-watson-ml.mybluemix.net/v3/wml_instances/f8b2d7ff-25d2-4ae6-bd84-ffafcda86acb/published_models/543c6a7c-fa0e-4341-8578-9f805750adfc/deployments/2fc0e621-7e04-4bcd-8bba-1ed8ee146094/online', json=payload_scoring, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))

Scoring response
{'fields': ['GENDER', 'AGE', 'MARITAL_STATUS', 'PROFESSION', 'features', 'rawPrediction', 'probability', 'prediction', 'nodeADP_class', 'nodeADP_classes'], 'values': [['M', 23, 'Married', 'Professional', [12, [0, 1, 2, 5], [2.0018477012437055, 2.271574273585683, 2.00045822401006, 2.80852940859655]], [2.0299441273450807, -2.0299441273450807], [0.8839053445872778, 0.11609465541272225], 0.0, 'FALSE', ['FALSE', 'TRUE']], ['F', 27, 'Single', 'Professional', [12, [1, 3, 5], [2.6666306689918886, 2.0353107937706234, 2.80852940859655]], [2.731323767581036, -2.731323767581036], [0.9388498798781167, 0.0611501201218833], 0.0, 'FALSE', ['FALSE', 'TRUE']]]}


### Display scoring results
As a final example, we will load the result scoring into a panda dataframe and display it with Brunel

In [4]:
# Create a panda DataFrame with the values and fields
import pandas as pd
jscore=json.loads(response_scoring.text)
dfsc=pd.DataFrame(data=jscore['values'],columns=jscore['fields'])
dfsc

,GENDER,AGE,MARITAL_STATUS,PROFESSION,features,rawPrediction,probability,prediction,nodeADP_class,nodeADP_classes
0,M,23,Married,Professional,"[12, [0, 1, 2, 5], [2.0018477012437055, 2.2715...","[2.0299441273450807, -2.0299441273450807]","[0.8839053445872778, 0.11609465541272225]",0.0,FALSE,"[FALSE, TRUE]"
1,F,27,Single,Professional,"[12, [1, 3, 5], [2.6666306689918886, 2.0353107...","[2.731323767581036, -2.731323767581036]","[0.9388498798781167, 0.0611501201218833]",0.0,FALSE,"[FALSE, TRUE]"


In [6]:
import brunel
%brunel data('dfsc') bar x(AGE) y(probability) color(GENDER)

<IPython.core.display.Javascript object>

Feel free to experiment with other visualizations or input data